In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [74]:
fake_data=pd.read_csv('~/Desktop/w207/Final_Project/w207-project/input/fake-and-real-news-dataset/Fake.csv')
true_data=pd.read_csv('~/Desktop/w207/Final_Project/w207-project/input/fake-and-real-news-dataset/True.csv')
fake_data['label']='fake'
true_data['label']='true'
fake_corpus=fake_data[['text','label']]
true_corpus=true_data[['text','label']]
frames=[fake_corpus,true_corpus]
data=pd.concat(frames)

train, test = train_test_split(data, test_size=0.2)

display(train)
display(test)


,text,label
18857,An elderly Jewish woman living in Paris was no...,fake
16321,HERE S A REMINDER OF JONATHAN GRUBER INSULTING...,fake
6526,The GOP implosion we have been waiting for has...,fake
5118,"While at a rally in Wilmington, North Carolina...",fake
3836,WASHINGTON (Reuters) - Senate Judiciary Commit...,true
...,...,...
6755,Republican presidential front-runner Donald Tr...,fake
19948,Scum of the earth isn t strong enough for thes...,fake
1042,Does Donald Trump have tapes of his conversati...,fake
14200,Obama s cowardly backdoor gun confiscation sta...,fake


,text,label
9475,Should legal gun owners be allowed to freely c...,fake
7763,Republican presidential candidate and Texas Se...,fake
9103,It s a rare occasion when a Hollywood legend d...,fake
6661,WASHINGTON (Reuters) - The FBI backs the CIA’s...,true
18048,Wow! This is one of the most powerful speeches...,fake
...,...,...
13415,GENEVA (Reuters) - The United Nations said on ...,true
5265,WASHINGTON (Reuters) - U.S. President Donald T...,true
13528,BEIRUT (Reuters) - Lebanon s Prime Minister Sa...,true
10125,"And once again, the snowflakes want to erase p...",fake


In [75]:
from sklearn.feature_extraction import text 
additional_words=['reuters','washington']
stop_words = text.ENGLISH_STOP_WORDS.union(additional_words)
print(stop_words)
print(train['label'].value_counts(),'\n Shape of Data is ',train.shape)
corpus=train['text']
labels=train['label']
test_corpus=test['text']
test_labels=test['label']
cv=CountVectorizer(ngram_range=(3,3),lowercase=True,token_pattern='\w{3,}',stop_words=stop_words)
train_cv=cv.fit_transform(corpus)
test_cv=cv.transform(test_corpus)


frozenset({'an', 'either', 'this', 'forty', 'rather', 'most', 'themselves', 'everything', 'latter', 'interest', 'nor', 'indeed', 'co', 'almost', 'sometimes', 'have', 'twelve', 'whereafter', 'except', 'would', 'ltd', 'fifteen', 'is', 'up', 'of', 'neither', 'your', 'made', 'some', 'thin', 'call', 'though', 'elsewhere', 'himself', 'again', 'eg', 'been', 'not', 'and', 'other', 'there', 'seemed', 'hereby', 'over', 'cant', 'for', 'he', 'even', 'whereupon', 'find', 'somehow', 'any', 'serious', 'moreover', 'empty', 'him', 'we', 'why', 'ourselves', 'still', 'somewhere', 'sometime', 'one', 'where', 'five', 'ever', 'twenty', 'thereupon', 'describe', 'whenever', 'amoungst', 'yet', 'bill', 'nine', 'seems', 'from', 'whole', 'could', 'perhaps', 'first', 'eight', 'she', 'here', 'full', 'ie', 'are', 'yourselves', 'three', 'get', 'nevertheless', 'all', 'may', 'otherwise', 'thereby', 'afterwards', 'can', 'via', 'latterly', 'detail', 'once', 'those', 'much', 'thereafter', 'de', 'a', 'same', 'which', 'belo

In [76]:
log_model=LogisticRegression(solver="liblinear", multi_class="auto")
log_model.fit(train_cv,labels)
pred_labels=log_model.predict(test_cv)
f1_score=round(metrics.f1_score(test_labels,pred_labels,average="weighted"),5)
weights=log_model.coef_
print('Logistic F1_score',f1_score)



Logistic F1_score 0.91701


In [77]:
knn_values=[1,3,5,7,9,11,13]
df_k_stats=pd.DataFrame()
knn_stats={'Model':'KNN','K-value':None,'F1-Score':None} #Create dictionary to hold KNN stats to append with dataframe
for i in knn_values:
    knn_stats['K-value'] = i #Store K-value within dictionary
    knn_model = KNeighborsClassifier(n_neighbors = i) #Create KNN model
    knn_model.fit(train_cv,labels) #Fit model on training data from CountVectorizer fit and transform
    knn_pred=knn_model.predict(test_cv) #Predict on dev dataset
    knn_model_f1=metrics.f1_score(test_labels,knn_pred,average="weighted") #Get f1-score based on the dev labels and predicted values
    knn_stats['F1-Score'] = round(knn_model_f1,4) #Make it pretty
    df_k_stats = df_k_stats.append(knn_stats, ignore_index=True) #place the values in dataframe
print('Show K1 and F1-Score for KNN Model','\n',df_k_stats.to_string(index=False,justify='center'))

Show K1 and F1-Score for KNN Model 
 Model  K-value  F1-Score
 KNN     1.0     0.4301 
 KNN     3.0     0.3778 
 KNN     5.0     0.3727 
 KNN     7.0     0.3715 
 KNN     9.0     0.3663 
 KNN    11.0     0.3650 
 KNN    13.0     0.3646 


In [78]:
df_alpha_stats=pd.DataFrame() #Create dataframe to hold alpha stats
alpha=[.00001,.0001,.001,.01,.1,1,2,5,10] #alpha-values for testing for Naive-Bayes model
for i in alpha:
    nb_stats={'Model':'Naive-Bayes','Alpha':None,'F1-Score':None} #Create dictionary to hold Naive-Bayes stats to append with dataframe
    nb_stats['Alpha']=i #Store alpha-value within dictionary
    nb_model=MultinomialNB(alpha=i) #Create Naive-Bayes model based on alpha value
    nb_model.fit(train_cv,labels) #Fit model on training data from CountVectorizer fit and transform
    nb_pred=nb_model.predict(test_cv) #Predict on dev dataset
    nb_model_f1=metrics.f1_score(test_labels,nb_pred,average="weighted") #Get f1-score based on the dev labels and predicted values
    nb_stats['F1-Score']=round(nb_model_f1,4) #Make it pretty
    df_alpha_stats=df_alpha_stats.append(nb_stats,ignore_index=True)  #place the values in dataframe
print('\nShow Alpha and F1-Score for Naive-Bayes Model','\n',df_alpha_stats.to_string(index=False,justify='center'))



Show Alpha and F1-Score for Naive-Bayes Model 
    Model      Alpha   F1-Score
Naive-Bayes  0.00001   0.9688 
Naive-Bayes  0.00010   0.9691 
Naive-Bayes  0.00100   0.9694 
Naive-Bayes  0.01000   0.9715 
Naive-Bayes  0.10000   0.9725 
Naive-Bayes  1.00000   0.9736 
Naive-Bayes  2.00000   0.9758 
Naive-Bayes  5.00000   0.9737 
Naive-Bayes 10.00000   0.9723 


In [79]:
weights=log_model.coef_ #Get the weights of the logistic regression
sorted_weights=np.argsort(weights)[:,-10:].flatten() #sort them in order to get the top 5 weights per topic
topic_words=cv.get_feature_names_out()[sorted_weights] #Grab the words from the top 5 weights 
categories=['true','fake']
topics=categories#set the topics to place in the dataframe
df_topics=pd.DataFrame() #create dataframe
df_topics['words']=pd.DataFrame(topic_words) #create the column of words for the top 20 words in each category
for i in range(1): #create for loop to go through the topics
    specific_weight=weights[i][sorted_weights] #Get the specific weight for that word
    df_topics[topics[i]]=pd.DataFrame(specific_weight,columns=[topics[i]]) #Place the word and topics in the row
display(df_topics) #showcase the topics and the words for the feature
y_pred_prob = log_model.predict_proba(test_cv)[:, 1]
print(y_pred_prob)


,words,true
0,president emmanuel macron,1.667915
1,white house said,1.703371
2,chancellor angela merkel,1.815570
3,state news agency,1.836994
4,president barack obama,1.838863
5,president tayyip erdogan,1.928007
6,told news conference,2.190746
7,president robert mugabe,2.206320
8,prime minister theresa,2.749074
9,president donald trump,3.711844


[0.00848072 0.10487506 0.15931626 ... 0.92878343 0.01243837 0.50925231]


In [80]:
weights=nb_model.coef_ #Get the weights of the logistic regression
sorted_weights=np.argsort(weights)[:,-10:].flatten() #sort them in order to get the top 5 weights per topic
topic_words=cv.get_feature_names_out()[sorted_weights] #Grab the words from the top 5 weights 
categories=['true','fake']
topics=categories#set the topics to place in the dataframe
df_topics=pd.DataFrame() #create dataframe
df_topics['words']=pd.DataFrame(topic_words) #create the column of words for the top 20 words in each category
for i in range(1): #create for loop to go through the topics
    specific_weight=weights[i][sorted_weights] #Get the specific weight for that word
    df_topics[topics[i]]=pd.DataFrame(specific_weight,columns=[topics[i]]) #Place the word and topics in the row
display(df_topics) #showcase the topics and the words for the feature


/Users/rohin/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


,words,true
0,russian president vladimir,-11.663681
1,speaker paul ryan,-11.657781
2,secretary state rex,-11.593243
3,state rex tillerson,-11.591406
4,president vladimir putin,-11.422942
5,president elect donald,-11.364234
6,elect donald trump,-11.361314
7,white house said,-11.270786
8,president barack obama,-10.123659
9,president donald trump,-9.446495
